In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import os
from tqdm import tqdm_notebook
import seaborn as sns
import re
from pprint import pprint

In [2]:
columns = ['entity', 'start_date', 'ProfitLoss',
        'ProfitLossFromOrdinaryActivitiesBeforeTax','ContributedCapital',
        'CurrentAssets', 'Equity', 'LiabilitiesAndEquity',
        'RetainedEarnings', 'ShorttermLiabilitiesOtherThanProvisions',
        'ClassOfReportingEntity',
        'AverageNumberOfEmployees',
        'AddressOfReportingEntityPostCodeIdentifier'
       ]

In [3]:
def preprocess_df(f):
    try:
        df = pd.read_csv(f, usecols=columns)
        company_class = df.ClassOfReportingEntity.unique()[0]
        zip_code = re.findall('\d{4}', str(df.AddressOfReportingEntityPostCodeIdentifier.unique()[0]))[0]
        df.drop(['ClassOfReportingEntity', 'AddressOfReportingEntityPostCodeIdentifier'], axis=1, inplace=True)
        df.start_date = df.start_date.apply(lambda x: str(x)[:4])
        # Aggregate the rows by the 
        df = df.groupby('start_date').mean()
        df['Target'] = df['Equity'].shift(-1)
        df['Class'] = company_class
        df['Zip_Code'] = zip_code
        df = df[~df.Target.isna()]
    except Exception as e:
       # print("Oups")
       # print(str(e))
        df = 0
    return df



In [4]:
files = ['Data/'+file for file in os.listdir('Data/') if file[-1]=='v']
files.extend(['New_data/'+file for file in os.listdir('New_data/') if file[-1]=='v'])

In [5]:
df = [preprocess_df(f) for f in tqdm_notebook(files)]

In [6]:
new_list = [dd for dd in df if type(dd) is not int]

In [7]:
new_df = pd.concat(new_list)

C:\Users\Bruger\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
# Turn Company type into either A, B, C or D 
def strip_company_type(x):
    try:
        return re.findall('[ABCD]', str(x))[0]
    except Exception as e:
        return np.NaN

In [9]:
new_df.Class = new_df.Class.apply(strip_company_type)

In [10]:
new_df = new_df.dropna()
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54250 entries, 2016 to 2017
Data columns (total 13 columns):
AverageNumberOfEmployees                     54250 non-null float64
Class                                        54250 non-null object
ContributedCapital                           54250 non-null float64
CurrentAssets                                54250 non-null float64
Equity                                       54250 non-null float64
LiabilitiesAndEquity                         54250 non-null float64
ProfitLoss                                   54250 non-null float64
ProfitLossFromOrdinaryActivitiesBeforeTax    54250 non-null float64
RetainedEarnings                             54250 non-null float64
ShorttermLiabilitiesOtherThanProvisions      54250 non-null float64
Target                                       54250 non-null float64
Zip_Code                                     54250 non-null object
entity                                       54250 non-null float64
dtypes: floa

In [15]:
new_df.to_pickle('2018_data_company_type_nb_employes_zip_code_ali.pkl')